In [1]:
# Подключаем нужные библиотеки
from kaggle_environments import make, evaluate
import numpy as np
import random
import collections
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# Обозначения для ходов:
# 0 - камень
# 1 - бумага
# 2 - ножницы

# Создаем окружение для игры с заданным количеством шагов (ходов)
env = make("rps", configuration={"episodeSteps": 100})

c:\Users\valer\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


termcolor not installed, skipping dependency
No pygame installed, ignoring import
Loading environment lux_ai_s3 failed: No module named 'chex'


Опишем поведение агента, всегда играющего "камень" - это значение 0

In [2]:
%%writefile rock_agent.py

#1
#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def rock_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


In [3]:
%%writefile paper_agent.py

#2
def paper_agent(observation, configuration):
    return 1

Overwriting paper_agent.py


In [4]:
%%writefile scissors_agent.py

#3
def scissors_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


In [5]:
%%writefile random_agent.py
import random

#4
# Агент выбирает случайное значение
def random_agent(observation, configuration):
    return random.randint(0,2)

Overwriting random_agent.py


In [6]:
%%writefile paper_scissors_agent.py
import random

#5
# Агент выбирает либо ножницы, любо бумагу
def paper_scissors_agent(observation, configuration):
    return random.randint(1,2)

Overwriting paper_scissors_agent.py


In [7]:
%%writefile rock_paper_agent.py
import random

#6
# Агент выбирает либо камень, либо бумагу
def rock_paper_agent(observation, configuration):
    return random.randint(0,1)

Overwriting rock_paper_agent.py


In [8]:
%%writefile rock_scissors_agent.py
import random

#7
# Агент выбирает либо камень, либо ножницы
def rock_scissors_agent(observation, configuration):
    pool = [0, 2]
    def random.choice(pool)

Overwriting rock_scissors_agent.py


In [9]:
%%writefile copy_opponent_agent.py
import random

#8
# Агент копирует последний ход опонента, если ход первый, выбирает случайно
def copy_opponent_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent_agent.py


In [10]:
%%writefile reactionary_agent.py
import random
import math

#9
# Агент реагирует на последнее действие опонента
def get_score(left_move, right_move):
    
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)
    
last_react_action = None
def reactionary_agent(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs
    
    return last_react_action

Overwriting reactionary_agent.py


In [11]:
%%writefile counter_reactironary_agent.py
import math
import random

#10
# Агент выбирает действие на основе результатов предыдущих ходов
def get_score(left_move, right_move):
    
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

last_counter_action = None
def counter_reactionary_agent(observation, configuration):
    global last_counter_action
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_counter_action

Overwriting counter_reactironary_agent.py


In [12]:
%%writefile statistical_agent.py
import random

#11
# Агент анализирует статистику ходов опонента
action_histogram = {}
def statistical_agent(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return random.randint(0, 2)
    
    action = observation.lastOpponentAction
    action_histogram[action] = action_histogram.get(action, 0) + 1

    mode_action = max(action_histogram, key=action_histogram.get)
    return (mode_action + 1) % configuration.signs

Overwriting statistical_agent.py


In [13]:
%%writefile Jonson_agent.py
#12
# Агент анализирирует паттерни опонента

import numpy as np
import collections
def Jonson_agent(observation, configuration):
    """
    Агент с анализом паттернов противника. Использует таблицу частот для предсказания.
    """
    k = 2
    global table, action_seq
    if observation.step % 25 == 0:
        action_seq, table = [], collections.defaultdict(lambda: [1, 1, 1])  # обновляет таблицу каждые 25 шагов
    if len(action_seq) <= 2 * k + 1:
        action = int(np.random.randint(3))
        if observation.step > 0:
            action_seq.extend([observation.lastOpponentAction, action])  # добавляет последний ход противника
        else:
            action_seq.append(action)
        return action

    key = ''.join([str(a) for a in action_seq[:-1]])  # ключ на основе последовательности ходов
    table[key][observation.lastOpponentAction] += 1  # обновляет таблицу по ключу

    action_seq[:-2] = action_seq[2:]
    action_seq[-2] = observation.lastOpponentAction
    key = ''.join([str(a) for a in action_seq[:-1]])
    
    if observation.step < 50:
        next_opponent_action_pred = np.argmax(table[key])  # предсказание по таблице
    else:
        scores = np.array(table[key])
        next_opponent_action_pred = np.random.choice(3, p=scores/scores.sum())
        
    action = (next_opponent_action_pred + 1) % 3
    if observation.step > 90:
        action = next_opponent_action_pred
    action_seq[-1] = action
    return int(action)

Overwriting Jonson_agent.py


In [14]:
%%writefile transition_agent.py
# 13. Агент с матрицей переходов для предсказания действий противника
import numpy as np
import pandas as pd
import random

T = np.zeros((3, 3))
P = np.zeros((3, 3))

a1, a2 = None, None

def transition_agent(observation, configuration):
    """
    Агент анализирует переходы между действиями противника и предсказывает следующее.
    """
    global T, P, a1, a2
    if observation.step > 1:
        a1 = observation.lastOpponentAction
        T[a2, a1] += 1  # обновляет матрицу переходов
        P = np.divide(T, np.maximum(1, T.sum(axis=1)).reshape(-1, 1))
        a2 = a1
        if np.sum(P[a1, :]) == 1:
            return int((np.random.choice([0, 1, 2], p=P[a1, :]) + 1) % 3)  # предсказание на основе вероятностей
        else:
            return int(np.random.randint(3))  # случайный ход
    else:
        if observation.step == 1:
            a2 = observation.lastOpponentAction
        return int(np.random.randint(3))

Overwriting transition_agent.py


In [23]:
# Турнир между агентами
agents = ['rock_agent', 'paper_agent', 'scissors_agent',
          'copy_opponent_agent', 'reactionary_agent', 
          'counter_reactironary_agent', 'statistical_agent',
          'Jonson_agent', 'rock_paper_agent', 'paper_scissors_agent',
          'rock_scissors_agent', 'transition_agent', 'random_agent']

# Добавляем .py к каждому агенту
agents_py = [i + ".py" for i in agents]
n_agents = len(agents)

# Инициализируем матрицу результатов
scores = np.zeros((n_agents, n_agents))

# Играем каждый агент против каждого
for i in range(n_agents):
    for j in range(i + 1, n_agents):
        result = evaluate("rps", [agents_py[i], agents_py[j]], configuration={"episodeSteps": 100}, num_episodes=1)
        scores[i][j] = result[0][0]  # сохраняем результат для агента i
        scores[j][i] = result[0][1]  # сохраняем результат для агента j

# Определяем победителя
vic = []
for i in range(n_agents):
    cnt_vic = sum(1 for j in range(n_agents) if scores[i][j] > 0)  # считаем победы агента
    vic.append(cnt_vic)

# Определяем индексы победителей
i_victors = [i for i, v in enumerate(vic) if v == max(vic)]
if len(i_victors) > 1:
    print("The winners are", '\n', *[agents_py[i] for i in i_victors], "!")
else:
    print("The winner is", agents_py[i_victors[0]], "!")

The winner is reactionary_agent.py !
